In [1]:
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [46]:
import torch

import pandas as pd
import numpy as np
import math

import nltk

from nltk.tokenize import word_tokenize, wordpunct_tokenize


%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
# import datasets

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 3.4 which is incompatible.
kaggle 1.5.8 requires urllib3<1.25,>=1.21.1, but you have urllib3 1.26.15 which is incompatible.
cloud-ml 0.0.1 requires requests<=2.25.1,>=2.22.0, but you have requests 2.28.2 which is incompatible.
cloud-ml 0.0.1 requires tqdm<=4.54.1,>=4.45.0, but you have tqdm 4.65.0 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
model_name = "ai-forever/ruRoberta-large"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense

In [48]:
#!c1.4


In [49]:
DATA_PATH = '/home/jupyter/mnt/s3/dataset_hackaton/'

In [50]:
!ls /home/jupyter/mnt/s3/dataset_hackaton/

bank-categories.csv
bank-sentiment.csv
banks-ethic-DB - NEW.docx
homework_shad_da.ipynb.txt
sentiment.csv
train.csv


In [51]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')

bank_categories_df = pd.read_csv(DATA_PATH + 'bank-sentiment.csv')
bank_sentiment_df = pd.read_csv(DATA_PATH + 'bank-categories.csv')


In [8]:
train_df

,Unnamed: 0,sentence,1category,2category,sentiment
0,4754,При этом всегда получал качественные услуги.,Communication,NaN,+
1,4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,NaN,−
2,3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,NaN,−
3,11640,Отвратительное отношение к клиентам.,Communication,NaN,−
4,5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,NaN,+
...,...,...,...,...,...
19356,8004,Никогда и ни в коем случае не открывайте счет ...,Communication,NaN,−
19357,18182,ТИ откровенно забили на качество и развивают с...,Quality,NaN,−
19358,744,"Я считаю, это прорыв и лидерство финансовых ус...",?,NaN,+
19359,6220,"Писал мужчина очень доходчиво, не финансовым я...",Communication,NaN,+


In [52]:
# indexes = np.arange(train_df.shape[0])
# np.random.shuffle(indexes)
# train_size = math.ceil(train_df.shape[0] * 0.8)

# train_indexes = indexes[:train_size]
# test_indexes = indexes[train_size:]

# train_data = train_df.iloc[train_indexes]
# test_data = train_df.iloc[test_indexes]

# train_data.reset_index(drop=True, inplace=True)
# test_data.reset_index(drop=True, inplace=True)


indexes = np.arange(train_df.shape[0])
np.random.shuffle(indexes)
train_size = math.ceil(train_df.shape[0] * 0.8)

data = train_df.iloc[indexes]

In [53]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples,
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
#     result['target'] = examples['target']
    return result

In [54]:
from torch.utils.data import Dataset, random_split


class Dataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: list, tokenizer):  
        self.data = data
        self.tokenizer = tokenizer

        self.feature_column = feature_column
        self.target_column = target_column
        self.label2num = lambda x: x == '+'

    def __getitem__(self, item):
        indx_tokens = preprocess_function(self.data[self.feature_column][item])
        label = self.label2num(self.data[self.target_column][item])
        
        return {**indx_tokens, "target": label}

    def get_tokens_(self, text):
          return self.tokenizer(text)

    def __len__(self):
        return self.data.shape[0]
    
    

In [55]:
dataset = Dataset(data, 'sentence', 'sentiment', tokenizer)

In [56]:
# dataset[0]

0

In [57]:
from torch.utils.data import DataLoader


batch_size = 8
num_workers = 4

def average_emb(batch):
    features = [b["tokens"] for b in batch]
    targets = [b["target"] for b in batch]

    return {"tokens": features, "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dataset) * 0.8)

# train, valid = random_split(dataset, [train_size, len(train_data) - train_size])
train, valid = random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=transformers.default_data_collator)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=transformers.default_data_collator)

In [15]:
# next(iter(train_loader))

In [58]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))[0]

{'input_ids': tensor([[    1, 47756, 15780,  ...,     0,     0,     0],
         [    1,   608,  6661,  ...,     0,     0,     0],
         [    1, 46718,  6722,  ...,     0,     0,     0],
         ...,
         [    1,  1229,  1058,  ...,     0,     0,     0],
         [    1, 35882,  2364,  ...,     0,     0,     0],
         [    1, 19951,    16,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([ True, False, False,  True, False,  True,  True, False])}

In [59]:
# newmodel.forward(input_ids=batch['input_ids'], attention_mask=batch['attention_mask']).last_hidden_state.shape

In [60]:
def freeze_layers(model, num_to_freeze=1):
    
    for layer in list(model.parameters())[:-num_to_freeze]:
        layer.requires_grad_ = False
    return model
        

In [61]:
class FBERTAcls(nn.Module):

    def __init__(self, model, last_layer_dim=768, hidden_layer_size=256):
        super().__init__()
        self.BERT = freeze_layers(model, 0)
        self.hidden_layer_1 = nn.Linear(last_layer_dim, hidden_layer_size)
        self.relu = nn.ReLU(inplace=True)
        self.bn_1 = nn.BatchNorm1d(hidden_layer_size)
        self.hidden_layer_2 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.bn_2 = nn.BatchNorm1d(hidden_layer_size)
        self.hidden_layer_3 = nn.Linear(hidden_layer_size, hidden_layer_size // 4)
        self.bn_3 = nn.BatchNorm1d(hidden_layer_size // 4)
        self.cls = nn.Linear(hidden_layer_size // 4, 1)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        bert_output = self.BERT(input_ids=input_ids, attention_mask=attention_masks)
        hidden_out = self.hidden_layer_1(bert_output.last_hidden_state[:, 0, :])

        out = self.bn_1(self.relu(hidden_out))
        out = self.bn_2(self.relu(self.hidden_layer_2(out)))
        out = self.bn_3(self.relu(self.hidden_layer_3(out)))

        logits = self.cls(out)
        return logits




In [62]:
model = FBERTAcls(newmodel, last_layer_dim=1024, hidden_layer_size=512)

In [63]:
device = 'cpu'

In [72]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}. Train Loss: {0}")
    
    model.train()
    
    total_loss = 0
    steps = 0
    for batch in pbar:
        optimizer.zero_grad()
        
        features = batch["input_ids"].to(device)
        attn_mask = bathc(['attention_mask'].t)
        
        targets = batch["targets"].to(device).reshape(-1, 1)

        predict = model(features)
        loss = criterion(predict, targets.to(torch.float32))
        loss.backward()

        optimizer.step()
        
        total_loss += loss.detach().cpu()
        
        pbar.set_description(f"Epoch {epoch + 1}. Train Loss: {loss.detach().cpu():.4}")
        steps += 1
        
    return total_loss / steps

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["input_ids"].to(device)
            targets = batch["targets"].to(device).reshape(-1, 1)
        
            predict_probs = model(features)
            loss = criterion(predict_probs, targets.to(torch.float32))

            predict = predict_probs > 0.5
            acc = (predict == targets).sum() / len(targets)

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

In [73]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

criterion = nn.BCEWithLogitsLoss()

In [74]:
for epoch in range(3):
    loss = training(model, optimizer, criterion, train_loader, epoch)
    
    
    

Epoch 1. Train Loss: 0:   0%|          | 0/1936 [00:00<?, ?it/s]

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-29-89483fdf7b66>:2 in <module>                                             │
│ <ipython-input-27-5a4c37499a59>:14 in training                                            │
│                                                                                           │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1051 in _call_impl      │
│                                                                                           │
│   1048 │   │   # this function, and just call forward.                                    │
│   1049 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1050 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1051 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1052 │   │   # Do not call functions when jit is used                                   │
│   1053 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1054 │   │   if self._backward_hooks or _global_backward_hooks:                         │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: forward() missing 1 required positional argument: 'token_type_ids'

TypeError: forward() missing 1 required positional argument: 'token_type_ids'

In [68]:
next(iter(train_loader))

{'input_ids': tensor([[    1,  4336,  8806,  ...,     0,     0,     0],
         [    1,  3667, 14679,  ...,     0,     0,     0],
         [    1, 11828,   281,  ...,     0,     0,     0],
         ...,
         [    1,  1133,  1118,  ...,     0,     0,     0],
         [    1, 23269,   503,  ...,     0,     0,     0],
         [    1,  2624, 46298,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([False, False, False,  True, False, False, False, False])}